In [1]:
class ElGamal:
    def __init__(self, size):
        def genPrime():
            q_size = 160
            q = random_prime(ZZ.random_element(2^(q_size-1),2^q_size-1))
            pi = 2*q + 1
            
            while not is_prime(pi) and len(pi.binary()) < self.size:
                q = next_prime(q)
                pi = 2*q + 1
                
            return pi,q
    
        def genParams():
            p,q = genPrime()
            R = GF(p)
            g = R.multiplicative_generator()
            return (p, q, g)
        
        
        
        self.size = size
        self.p, self.q, self.g = genParams()
        
    
    def keyGen(self):
        a = ZZ.random_element(2, (self.q)-1)
        beta = power_mod(self.g, a, self.p)
        return a, beta
    
    def KEM(self, beta, r=None):
        if r is None:
            r = ZZ.random_element(2, (self.q)-1)
        
        key = pow(beta, r, self.p)
        enc = pow(self.g, r, self.p)
        return key, enc
    
    def KRev(self,a ,enc):
        return pow(enc, a, self.p)
        
    

In [2]:
# Generation of the keys
eg = ElGamal(1024)
alice_pvk, alice_pbk = eg.keyGen()
bob_pvk, bob_pbk = eg.keyGen()
print("Alice's keys: ", alice_pvk, alice_pbk)
print("Bob's keys: ", bob_pvk, bob_pbk)

Alice's keys:  132613159081623024258031172495203335620918608161 360248410515475520909758006695331845491369648308
Bob's keys:  127112148370600925241131131749795095790615945179 415059872375246466298304934168944568195030201572


In [3]:
# Sharing of the keys

alice_key, alice_enc = eg.KEM(bob_pbk)
print("Alice: ", alice_key)
bob_key = eg.KRev(bob_pvk, alice_enc)
print("Bob: ", bob_key)


Alice:  242847917013476179086241583816579197350737405925
Bob:  242847917013476179086241583816579197350737405925


In [4]:
import hashlib
import os

In [5]:
def xor(b, a):
    return bytes([a^^b for a,b in zip(a,b)])

In [6]:
class FO_ElGamal:
    def __init__(self, size):
        self.kem = ElGamal(size)
    
    def encrypt(self,m, key):
        x = m.encode()
        
        r = ZZ.random_element(2, (self.kem.q)-1)
        r = str(r).encode()
        # Cipher r
        gr = hashlib.sha256(r).digest()
        # XOR
        y = xor(x,gr)
        # Concat r and y
        yr = y + r
        yr = int.from_bytes(yr, byteorder='big')
        # KEM
        k, e = self.kem.KEM(key, yr)
       
        k_ = int(k).to_bytes(len(r), byteorder='big')
        
        #k_ = bytes(str(Integer(k).binary()).encode())
        # XOR k and r
        c = xor(k_,r)
        
        return y,e,c
    
    def decrypt(self, y, e, c, pvk, pbk):
        # KRev
        k = self.kem.KRev(pvk, e)
        k_ = int(k).to_bytes(len(c), byteorder='big')
        
        # XOR k and c
        r = xor(k_,c)
        
        # Check if the decryption can be done
        yr = y + r
        yr = int.from_bytes(yr, byteorder='big')
        
        
        if (k,e) != (self.kem.KEM(pbk, yr)):
            return "Decryption failed"
        
        # Decryption
        g = hashlib.sha256(r).digest()
        pt = xor(g,y)
        
        return pt.decode()
   
        
        
        
        
    

In [7]:
fo = FO_ElGamal(1024)

# Gen Keys
apvk, apbk = fo.kem.keyGen()
bpvk, bpbk = fo.kem.keyGen()

In [8]:
y,e,c = fo.encrypt("TP2 Fujisaki-Okamoto", bpbk)
pt = fo.decrypt(y,e,c, bpvk, bpbk)
pt

'TP2 Fujisaki-Okamoto'